### Carregando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import sklearn
import json
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics

from sklearn.linear_model import LogisticRegression

### Importanto e tratando os datasets de cada pais

In [2]:
CAvideos = pd.read_csv('CAvideos.csv')
DEvideos = pd.read_csv('DEvideos.csv')
FRvideos = pd.read_csv('FRvideos.csv')
GBvideos = pd.read_csv('GBvideos.csv')
USvideos = pd.read_csv('USvideos.csv')

In [3]:
print("CA DataSet shape: "+str(CAvideos.shape))
print("DE DataSet shape: "+str(DEvideos.shape))
print("FR DataSet shape: "+str(FRvideos.shape))
print("GB DataSet shape: "+str(GBvideos.shape))
print("US DataSet shape: "+str(USvideos.shape))

CA DataSet shape: (40881, 16)
DE DataSet shape: (40840, 16)
FR DataSet shape: (40724, 16)
GB DataSet shape: (38916, 16)
US DataSet shape: (40949, 16)


In [4]:
#print("CA DataSet null inputs: \n"+str(CAvideos.isnull().sum())+"\n")
#print("DE DataSet null inputs: \n"+str(DEvideos.isnull().sum())+"\n")
#print("FR DataSet null inputs: \n"+str(FRvideos.isnull().sum())+"\n")
#print("GB DataSet null inputs: \n"+str(GBvideos.isnull().sum())+"\n")
print("US DataSet null inputs: \n"+str(USvideos.isnull().sum())+"\n")

US DataSet null inputs: 
video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
thumbnail_link              0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
description               570
dtype: int64



In [5]:
CAvideos = CAvideos.dropna(axis=0)
DEvideos = DEvideos.dropna(axis=0)
FRvideos = FRvideos.dropna(axis=0)
GBvideos = GBvideos.dropna(axis=0)
USvideos = USvideos.dropna(axis=0)

In [6]:
#print("CA DataSet null inputs: \n"+str(CAvideos.isnull().sum())+"\n")
#print("DE DataSet null inputs: \n"+str(DEvideos.isnull().sum())+"\n")
#print("FR DataSet null inputs: \n"+str(FRvideos.isnull().sum())+"\n")
#print("GB DataSet null inputs: \n"+str(GBvideos.isnull().sum())+"\n")
print("US DataSet null inputs: \n"+str(USvideos.isnull().sum())+"\n")

US DataSet null inputs: 
video_id                  0
trending_date             0
title                     0
channel_title             0
category_id               0
publish_time              0
tags                      0
views                     0
likes                     0
dislikes                  0
comment_count             0
thumbnail_link            0
comments_disabled         0
ratings_disabled          0
video_error_or_removed    0
description               0
dtype: int64



In [7]:
#print("CA DataSet shape: "+str(CAvideos.shape))
#print("DE DataSet shape: "+str(DEvideos.shape))
#print("FR DataSet shape: "+str(FRvideos.shape))
#print("GB DataSet shape: "+str(GBvideos.shape))
print("US DataSet shape: "+str(USvideos.shape))

US DataSet shape: (40379, 16)


## Correção de formato de data-hora para padrão Unix

In [8]:
USvideos['trending_date'] = pd.to_datetime(USvideos['trending_date'], format='%y.%d.%m')
USvideos['publish_time'] = pd.to_datetime(USvideos['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# separates date and time into two columns from 'publish_time' column

USvideos.insert(4, 'publish_date', USvideos['publish_time'].dt.date)
USvideos['publish_time'] = USvideos['publish_time'].dt.time
USvideos['publish_date']=pd.to_datetime(USvideos['publish_date'])

In [9]:
USvideos.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,2017-11-13,22,17:13:01,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,2017-11-13,24,07:30:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12,23,19:05:24,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,2017-11-13,24,11:00:04,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,2017-11-12,24,18:01:41,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


## Tratando para pegar somente a ultima entrada para cada video (a entrada mais atualizada)

In [10]:
USvideos_lastentry = USvideos.sort_values('trending_date', ascending=False).drop_duplicates(['video_id'])
USvideos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr..."
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...


In [11]:
print("US DataSet shape: "+str(USvideos_lastentry.shape))

US DataSet shape: (6254, 17)


## Buscando o nome da categoria pelo identificador e armazenando em nova coluna

In [12]:
with open('US_category_id.json') as f:
    UScategory = json.load(f)

In [13]:
def category_replace(c_id):
    for i in UScategory["items"]:
        if int(c_id) == int(i["id"]):
             return i["snippet"]["title"]
    return "None"

In [14]:
USvideos_lastentry['category_name'] = USvideos_lastentry.apply(lambda row: category_replace(row['category_id']), axis=1)

In [15]:
USvideos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...,Gaming
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...,News & Politics
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr...",Film & Animation
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...,Howto & Style
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...,Music


## Normalização das features numéricas

In [16]:
USvideos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count
count,6254.000000,6.254000e+03,6.254000e+03,6.254000e+03,6.254000e+03
mean,20.408539,1.975222e+06,5.592469e+04,2.828081e+03,6.289861e+03
std,7.256811,7.092548e+06,1.942315e+05,2.433277e+04,2.943713e+04
min,1.000000,5.590000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,1.648875e+05,2.931000e+03,1.300000e+02,3.870000e+02
50%,24.000000,5.270375e+05,1.222400e+04,4.485000e+02,1.287500e+03
75%,25.000000,1.490734e+06,3.898500e+04,1.516000e+03,4.110750e+03
max,43.000000,2.252119e+08,5.613827e+06,1.643059e+06,1.228655e+06


In [17]:
views_mean = USvideos_lastentry['views'].mean()
views_std = USvideos_lastentry['views'].std()
USvideos_lastentry['views'] = (USvideos_lastentry['views']-views_mean)/views_std
USvideos_lastentry['likes'] = (USvideos_lastentry['likes']-USvideos_lastentry['likes'].mean())/USvideos_lastentry['likes'].std()
USvideos_lastentry['dislikes'] = (USvideos_lastentry['dislikes']-USvideos_lastentry['dislikes'].mean())/USvideos_lastentry['dislikes'].std()
USvideos_lastentry['comment_count'] = (USvideos_lastentry['comment_count']-USvideos_lastentry['comment_count'].mean())/USvideos_lastentry['comment_count'].std()


In [18]:
USvideos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count
count,6254.000000,6.254000e+03,6.254000e+03,6.254000e+03,6.254000e+03
mean,20.408539,-9.089130e-18,-9.089130e-18,6.816847e-18,-4.544565e-18
std,7.256811,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,1.000000,-2.784138e-01,-2.879280e-01,-1.162252e-01,-2.136710e-01
25%,17.000000,-2.552446e-01,-2.728377e-01,-1.108826e-01,-2.005243e-01
50%,24.000000,-2.041840e-01,-2.249928e-01,-9.779327e-02,-1.699337e-01
75%,25.000000,-6.830948e-02,-8.721392e-02,-5.392240e-02,-7.402592e-02
max,43.000000,3.147482e+01,2.861483e+01,6.740831e+01,4.152460e+01
